In [2]:
import cv2 as cv
import mediapipe as mp
import time
import math
import webbrowser
import numpy as np
import json
import os
import pyautogui
import tkinter as tk
import requests
from tkinter import *
from PIL import Image, ImageTk
from pynput import mouse, keyboard
from pynput.mouse import Button
from pynput.keyboard import Key

# variables
CEF_COUNTER = 0
eye_side = ''
closeEyes = False
eye_counter = 0
scrollFrameX = 0
scrollFrameY = 0
distance_counter = 0
startTimeInterval = 0
eyeRatioValues = np.array([])
calibrated = False
caliTimer = 120
settingWindow = []
smoothMouse = [0, 0]

rollRatio = 0
clickingMode = 1
timeUse = 0

# constants
#EYERATIO = 5
ROLLRATIO = 0.25
CLOSED_EYES_FRAME = 6
FONTS = cv.FONT_HERSHEY_COMPLEX
wCam, hCam = 640, 480
frameR = 100  # Frame Reduction
moveSoftening = 0.20
scrollSoftening = 100
face_record_area = 800
#distance_threshold = 0.2
distanceMessage = "You have been sitting too close to the computer."
blinkMessage = "You have been blinking less than 6 blinks per minute."
WELCOME_DURATION = 4000

# face bounder indices 
FACE_OVAL = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]

# Left eyes indices
LEFT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]

# right eyes indices
RIGHT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

#Configable
#token = "{LINE NOTIFY TOKEN HERE}"
#work_time = 55
#rest_time = 5
config = {
    "work_time" : 55,
    "rest_time" : 5,
    "distance_threshold" : 0.2,
    "blinking_threshold" : 6,
    "token" : "{LINE NOTIFY TOKEN HERE}",
    "EYERATIO" : 5,
    "start1" : [wCam // 2 - 50, hCam // 2 - 100],
    "end1" : [wCam // 2 + 50, hCam // 2],
    "start2" : [wCam // 2 - 150, hCam // 2 - 100],
    "end2" : [wCam // 2 + 150, hCam // 2]
}

if os.path.exists("config.txt"):
    with open("config.txt", "r") as file:
        try:
            config = json.load(file)
        except:
            pass
else:
    with open("config.txt", "w") as outfile:
        json.dump(config, outfile)

#print(config)
#start1 = (wCam // 2 - 50, hCam // 2 - 100)
#end1 = (wCam // 2 + 50, hCam // 2)
#start2 = (wCam // 2 - 150, hCam // 2 - 100)
#end2 = (wCam // 2 + 150, hCam // 2)

map_face_mesh = mp.solutions.face_mesh
face_detection = mp.solutions.face_detection
MODEL_SELECTION = 1
GREEN = (0, 255, 0)
RED = (0, 0, 255)
BLUE = (255, 0, 0)
PURPLE = (255, 0, 255)

Mouse = mouse.Controller()
Keyboard = keyboard.Controller()

# camera object
camera = cv.VideoCapture(0)
camera.set(3, wCam)
camera.set(4, hCam)

SCROLL_MODE = False
PROG_MODE = True
FRAME_SET_MODE = False
wScr, hScr = pyautogui.size() #1920*1080, 800, 600
clocx, clocy = 0, 0
plocx, plocy = 0, 0
mousex, mousey = round(wScr / 2), round(hScr / 2)
length = 20
pTime = 0
fps = 0

main_win = tk.Tk()

screen_width = main_win.winfo_screenwidth()
screen_height = main_win.winfo_screenheight()

#window size
app_width = int(screen_width/3)
app_height = int(screen_height/3)

#main_win.geometry(f'{app_width}x{app_height}+{int((screen_width-app_width)/2)}+{screen_height-app_height-80}')
main_win.geometry(f'+{0}+{100}')
main_win.title("AccessiblePyControl")
main_win.iconbitmap('logo.ico')

main_win.counter = 0
main_win.counter_close = 0
main_win.counter_time = 0
main_win.counter_dis = 0
nextStatus = 'rest'

focal_length = 1 * wCam
cam_matrix = np.array([ [focal_length, 0, wCam / 2],
                        [0, focal_length, hCam / 2],
                        [0, 0, 1]])
dist_matrix = np.zeros((4, 1), dtype=np.float64)

class Blink():
    def __init__(self):
        self.lastCount = 0
        self.blinkPerMinute = 4
        self.alert = False
    def startCount(self, startTime):
        self.lastCount = startTime
    def blinking(self, nowTime):
        timePass = nowTime - self.lastCount
        self.blinkPerMinute = self.blinkPerMinute * (5 / 6) + timePass / 6
        self.startCount(nowTime)
    def checkEnd(self):
        blinkPerMinute = 60 / self.blinkPerMinute
        if (blinkPerMinute <= config["blinking_threshold"]):
            if (not self.alert):
                lineNotifyMessage(blinkMessage, config["token"])
                self.alert = True
        else:
            self.alert = False
        #L5['text'] = "Blink per Minute: %.2f" % (blinkPerMinute)

def EMA(ValueToday, EMAYesterday, smoothingRatio):
    EMAToday=( ValueToday*(2/ (1+smoothingRatio)) )+( EMAYesterday*(1-(2/(1+smoothingRatio))) )
    return (EMAToday)
        
def Warning(text):
    top = Toplevel()
    top.title('Warning')
    Message(top, text=text, padx=20, pady=20).pack()
    stay_on_top(top)
    top.after(WELCOME_DURATION, top.destroy)
    
def closeSetting():
    global settingWindow
    settingWindow[0].destroy()
    settingWindow = []
    
def Setting():
    if (len(settingWindow) == 0):
        settingWindow.append(Toplevel())
        top = settingWindow[0]
        top.title('Setting')
        top.protocol("WM_DELETE_WINDOW", closeSetting)
        tk.Label(top, text="Working time").grid(row=0, pady=10)
        tk.Label(top, text="Resting time").grid(row=1, pady=20)
        tk.Label(top, text="Minimum distance").grid(row=2, pady=10)
        tk.Label(top, text="Minimun blinking").grid(row=3, pady=20)
        tk.Label(top, text="Line Notify token").grid(row=4, pady=10)
        L_error = tk.Label(top, text="The inputs are incorrect.")
        e1 = tk.Entry(top)
        e2 = tk.Entry(top)
        e3 = tk.Entry(top)
        e4 = tk.Entry(top)
        e5 = tk.Entry(top)
        e1.insert(0, config["work_time"])
        e2.insert(0, config["rest_time"])
        e3.insert(0, config["distance_threshold"])
        e4.insert(0, config["blinking_threshold"])
        e5.insert(0, config["token"])
        e1.grid(row=0, column=1)
        e2.grid(row=1, column=1)
        e3.grid(row=2, column=1)
        e4.grid(row=3, column=1)
        e5.grid(row=4, column=1)
        tk.Label(top, text="minutes").grid(row=0, column=2)
        tk.Label(top, text="minutes").grid(row=1, column=2)
        tk.Label(top, text="meters").grid(row=2, column=2)
        tk.Label(top, text="blink/minute").grid(row=3, column=2)
        tk.Button(top, text="Save", command=lambda : updateSetting(e1, e2, e3, e4, e5, L_error)).grid(row=5, column=3, padx=10)
        tk.Button(top, text="Exit", command=closeSetting).grid(row=0, column=3, padx=10)
        #Message(top, text=text, padx=20, pady=20).pack()
        stay_on_top(top)
        
        #top.after(WELCOME_DURATION, top.destroy)
    else:
        settingWindow[0].lift()

def is_integer(n):
    try:
        float(n)
    except ValueError:
        return False
    return True
    
def updateSetting(e1, e2, e3, e4, e5, L_error):
    global config
    Working_time = e1.get()
    Resting_time = e2.get()
    Minimum_distance = e3.get()
    Minimun_blinking = e4.get()
    LineNotifytoken = e5.get()
    if (is_integer(Working_time) and is_integer(Resting_time) and is_integer(Minimum_distance) and \
        is_integer(Minimun_blinking) and len(LineNotifytoken) > 0):
        config = {
            "work_time" : float(Working_time),
            "rest_time" : float(Resting_time),
            "distance_threshold" : float(Minimum_distance),
            "blinking_threshold" : float(Minimun_blinking),
            "token" : LineNotifytoken,
            "EYERATIO" : config["EYERATIO"],
            "start1" : config["start1"],
            "end1" : config["end1"],
            "start2" : config["start2"],
            "end2" : config["end2"]
        }
        with open("config.txt", "w") as outfile:
            json.dump(config, outfile)
        L_error.grid_remove()
    else:
        L_error.grid(row=5, columnspan=4, pady=10)
    
def eyeCalibrate():
    global caliTimer, eyeRatioValues
    caliTimer = 0
    eyeRatioValues[:] = 0

def landmarksDetection(results):
    mesh_coord = [(int(point.x * wCam), int(point.y * hCam), point.z) for point in
                  results.multi_face_landmarks[0].landmark]
    return mesh_coord

def euclaideanDistance(point, point1):
    x, y = point[:2]
    x1, y1 = point1[:2]
    distance = math.sqrt((x1 - x) ** 2 + (y1 - y) ** 2)
    return distance

def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def blinkRatio(landmarks, right_indices, left_indices):
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]
    # vertical line
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]
    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)
    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)
    if (rvDistance != 0):
        reRatio = rhDistance / rvDistance
    else:
        reRatio = 10
    if (lvDistance != 0):
        leRatio = lhDistance / lvDistance
    else:
        leRatio = 10
    
    ratio = (reRatio + leRatio) / 2
    return ratio, reRatio, leRatio

def numSign(num):
    if (num > 0):
        return (1)
    if (num < 0):
        return (-1)
    return (0)

def on_key():
    global MODEL_SELECTION
    if (MODEL_SELECTION == 1):
        MODEL_SELECTION = 2
    elif (MODEL_SELECTION == 2):
        MODEL_SELECTION = 1
    Keyboard.press(Key.cmd)
    Keyboard.press(Key.ctrl)
    Keyboard.press("o")
    Keyboard.release(Key.cmd)
    Keyboard.release(Key.ctrl)
    Keyboard.release("o")

def on_pro():
    global PROG_MODE
    PROG_MODE = not PROG_MODE

#def clicked():
    #main_win.counter += 1
    #L['text'] = 'Total Blink:' + str(main_win.counter)

#def closed():
    #main_win.counter_close += 1
    #L1['text'] = 'Total Closed Eyes:' + str(main_win.counter_close)

def switchClickingMode():
    global clickingMode
    if (clickingMode == 1):
        clickingMode = 2
        #L6['text'] = "Clicking Mode: Head"
    else:
        clickingMode = 1
        #L6['text'] = "Clicking Mode: Eye"
    
def updateClock(seconds, frame):
    global nextStatus, startTimeInterval
    
    if (nextStatus == 'rest'):
        remain_time =  config["work_time"] * 60 - (seconds - startTimeInterval)
    elif (nextStatus == 'work'):
        remain_time =  config["rest_time"] * 60 - (time.time() - startTimeInterval)
    if (remain_time < 0 and nextStatus == 'rest'):
        #print('rest')
        nextStatus = 'work'
        startTimeInterval = time.time()
        remain_time = 0
        Warning('Take a Rest')
    elif(remain_time < 0  and nextStatus == 'work'):
        #print('finish rest')
        nextStatus = 'rest'
        startTimeInterval = seconds
        remain_time = 0
        Warning('finish rest')
        
    seconds = remain_time % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60

    ac_t = "%d:%02d:%02d" % (hour, minutes, seconds)
    cv.putText(frame, 'Time to ' + nextStatus + ': ' + ac_t, (10, 60), FONTS, 1, BLUE, 2)

def distance(mesh_coords):
    global distance_counter, faceColor
    
    face_area = PolyArea(np.array([mesh_coords[p] for p in FACE_OVAL ])[:,0],np.array([mesh_coords[p] for p in FACE_OVAL ])[:,1])
    Distance = math.sqrt(face_record_area / face_area)
    #L3['text'] = 'Distance: %.2f'%Distance
    if (Distance < config["distance_threshold"]):
        faceColor = RED
    else:
        faceColor = GREEN
    if (faceColor == RED):
        distance_counter += 1
    else:
        distance_counter = 0
    if (distance_counter == 60):
        lineNotifyMessage(distanceMessage, config["token"])
                
def setup():
    L['text'] = 'Total Blink: 0'
    L1['text'] = 'Total Closed Eyes: 0'
    updateClock(config["work_time"] * 60)
    L3['text'] = 'Distance: 0'
    
def changeText(L4, mode):
    L4['text'] = mode
    L4.grid(row=2, column=0,columnspan=7, pady=1, padx=1)
    L4.after(5000, L4.grid_remove)

def stay_on_top(win):
    win.attributes('-topmost', True)
    win.update()
    
def update_mouse_freestyle(clocx, clocy):
    global mousex
    global mousey
    
    if (clocx > mousex + length):
        mousex = clocx - length
    elif (clocx < mousex - length):
        mousex = clocx + length
    if (clocy > mousey + length):
        mousey = clocy - length
    elif (clocy < mousey - length):
        mousey = clocy + length
        
def lineNotifyMessage(msg, token):
    try:
        headers = {
            "Authorization": "Bearer " + token, 
            "Content-Type" : "application/x-www-form-urlencoded"
        }

        payload = {'message': msg}
        r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    except:
        pass
        
def on_closing():
    try:
        face_mesh.close()
    except:
        pass
    try:
        camera.release()
    except:
        pass
    main_win.destroy()
    print('close')
    
def newFramePosition(face_mesh, camera, hPw):
    ret, frame = camera.read()  # getting frame from camera
    if not ret:
        changeText(L4, "Cannot set frame positon.")
        FRAME_SET_MODE = False
        main_win.after(1, frameUpdate)
        return (None)
    frame = cv.flip(frame, 1)
    rgb_frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
    try:
        results = face_mesh.process(rgb_frame)
    except:
        changeText(L4, "Cannot set frame positon.")
        FRAME_SET_MODE = False
        main_win.after(1, frameUpdate)
        return (None)
    if results.multi_face_landmarks:
        mesh_coords = landmarksDetection(results)
        pos = mesh_coords[9]
        dis = euclaideanDistance(mesh_coords[LEFT_EYE[0]], mesh_coords[RIGHT_EYE[8]])

        config["start1"] = (round(pos[0] - dis), round(pos[1] - dis))
        config["end1"] = (round(pos[0] + dis), round(pos[1] + dis))
        config["start2"] = (round(pos[0] - dis * 5), round(pos[1] - dis * 5 * hPw))
        config["end2"] = (round(pos[0] + dis * 5), round(pos[1] + dis * 5 * hPw))

def RollAngle(forehead, chin):
    deltaX = forehead[0] - chin[0]
    deltaY = forehead[1] - chin[1]
    try:
        slope = deltaX / deltaY
    except:
        slope = deltaX / 1
    return (slope)

def open_link():
    webbrowser.open("https://github.com/yoshi151/AccessiblePyControl/")
    #lnk1.grid(row = 1,rowspan=3, column=4 ,sticky= 'w')

def open_link2():
    webbrowser.open("https://github.com/yoshi151/AccessiblePyControl/blob/main/README.md")
        
frame = np.random.randint(0, 255, [100, 100, 3], dtype='uint8')
img = ImageTk.PhotoImage(Image.fromarray(frame))
#crop image from camera
button_image = tk.Label(main_win, width=int(app_width),image = img)
button_image.grid(row=0, column=0, columnspan=7, pady=1, padx=1)

photo=Image.open("set.png")
photo= photo.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo= ImageTk.PhotoImage(photo)
photo1=Image.open("cal.png")
photo1= photo1.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo1= ImageTk.PhotoImage(photo1)
photo2=Image.open("swi.png")
photo2= photo2.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo2= ImageTk.PhotoImage(photo2)
photo3=Image.open("fra.png")
photo3= photo3.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo3= ImageTk.PhotoImage(photo3)
photo4=Image.open("key.png")
photo4= photo4.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo4= ImageTk.PhotoImage(photo4)
photo5=Image.open("man.png")
photo5= photo5.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo5= ImageTk.PhotoImage(photo5)
photo6=Image.open("exi.png")
photo6= photo6.resize((int(app_width/11),int(app_width/11)), Image.Resampling.LANCZOS)
photo6= ImageTk.PhotoImage(photo6)


b = tk.Button(main_win,image=photo,text="Setting",font=('Arial',int(app_width/50)),height=int(app_width/9), command=Setting,compound=BOTTOM)
b.grid(row=1, column=0, sticky='we', pady=1, padx=1)
b1 = tk.Button(main_win,image=photo1,text="Calibrate",font=('Arial',int(app_width/50)),height=int(app_width/9), command=eyeCalibrate,compound=BOTTOM)
b1.grid(row=1, column=1, sticky='we', pady=1, padx=1)
b2 = tk.Button(main_win,image=photo2,text="Set Mode",font=('Arial',int(app_width/50)),height=int(app_width/9), command=switchClickingMode,compound=BOTTOM)
b2.grid(row=1, column=2, sticky='we', pady=1, padx=1)
b3 = tk.Button(main_win,image=photo3,text="Set Frame",font=('Arial',int(app_width/50)),height=int(app_width/9), command=lambda: newFramePosition(face_mesh, camera, hScr / wScr),compound=BOTTOM)
b3.grid(row=1, column=3, sticky='we', pady=1, padx=1)
b4 = tk.Button(main_win,image=photo4,text="Keyboard",font=('Arial',int(app_width/50)),height=int(app_width/9), command=on_key,compound=BOTTOM)
b4.grid(row=1, column=4, sticky='we', pady=1, padx=1)
b5 = tk.Button(main_win,image=photo5,text="Manual",font=('Arial',int(app_width/50)),height=int(app_width/9), command=open_link,compound=BOTTOM)
b5.grid(row=1, column=5, sticky='we', pady=1, padx=1)
b6 = tk.Button(main_win,image=photo6,text="Exit",font=('Arial',int(app_width/50)),height=int(app_width/9), command=on_pro,compound=BOTTOM)
b6.grid(row=1, column=6, sticky='we', pady=1, padx=1)

L4 = Label(main_win, text="", bg='#fff', fg='#00f',font=('Arial',int(app_width/45)))

# button_1 = tk.Button(main_win, text="keyboard", command=on_key, height=2, width=13)
# button_1.grid(row=0, column=0, rowspan=4, pady=1, padx=1)
# L = Label(main_win, text="Total Blink:", bg='#fff', fg='#f00')
# L.grid(row=0, column=1, pady=0, padx=0)
# L1 = Label(main_win, text="Total Closed Eyes:", bg='#fff', fg='#00f')
# L1.grid(row=1, column=1, pady=0, padx=0)
# L2 = Label(main_win, text="Used Time:", bg='#fff', fg='#00f')
# L2.grid(row=2, column=1, pady=0, padx=0)
# L3 = Label(main_win, text="Distance:", bg='#fff', fg='#00f')
# L3.grid(row=3, column=1, pady=0, padx=0)
# L4 = Label(main_win, text="", bg='#fff', fg='#00f')
# L4.grid(row=6, column=1, pady=0, padx=0)
# L5 = Label(main_win, text="Blink per Minute:", bg='#fff', fg='#00f')
# L5.grid(row=4, column=1, pady=0, padx=0)

# button_2 = tk.Button(main_win, text="Exit", command=on_pro, height=2, width=13)
# button_2.grid(row=0, column=2, rowspan=4, pady=1, padx=1)
# setFrameButton = tk.Button(main_win, text="Set Frame", command=lambda : newFramePosition(face_mesh, camera, hScr / wScr), height=2, width=13)
# setFrameButton.grid(row=6, column=2, rowspan=4, pady=1, padx=1)
# button_image = tk.Label(main_win)
# button_image.grid(row=5, column=0, columnspan=3, pady=1, padx=10)
# button_3 = tk.Button(main_win, text="Eye", command=eyeCalibrate, height=2, width=4)
# button_3.grid(row=0, column=3, rowspan=8, pady=1, padx=1)
# button_3 = tk.Button(main_win, text="Switch", command=switchClickingMode, height=2, width=4)
# button_3.grid(row=6, column=1, rowspan=8, pady=1, padx=1)
# button_setting = tk.Button(main_win, text="Setting", command=Setting, height=2, width=4)
# button_setting.grid(row=6, column=3, rowspan=8, pady=1, padx=1)
# L6 = Label(main_win, text="Clicking Mode: Eye", bg='#fff', fg='#00f')
# L6.grid(row=6, column=0, pady=0, padx=0)

blinkTimer = Blink()
start_time = time.time()
time_pass = 0
now_clock = start_time
blinkTimer.startCount(0)
first_out = 0
#setup()
stay_on_top(main_win)

face_mesh = map_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)
# starting Video loop here.
def frameUpdate():
    global first_out, time_pass, now_clock, eye_side, CEF_COUNTER, SCROLL_MODE, pTime, scrollFrameX,\
    scrollFrameY, eyeRatioValues, caliTimer, eye_counter, closeEyes, smoothMouse, fps
    
    ret, frame = camera.read()  # getting frame from camera
    if not ret:
        on_closing()
        return (1)
    frame = cv.flip(frame, 1)
    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    rgb_frame.flags.writeable = False
    # create mouse pad frame

    ############# MESH ##################
    try:
        results = face_mesh.process(rgb_frame)
    except:
        on_closing()
        return (1)
    if results.multi_face_landmarks:
        if (first_out):
            time_pass += time.time() - now_clock
            first_out = 0
        now_clock = time.time()
        timeUse = now_clock - start_time - time_pass
        updateClock(timeUse, frame)
        mesh_coords = landmarksDetection(results)
        distance(mesh_coords)
        nose = mesh_coords[1]
        nose_2d = (nose[0], nose[1])
        nose_3d = (nose[0], nose[1], nose[2] * 8000)
        face_2d = []
        face_3d = []
        for i in [33, 263, 1, 61, 291, 199]:
            p = mesh_coords[i]
            x, y = int(p[0]), int(p[1])
            face_2d.append([x, y])
            face_3d.append([x, y, p[2]])
        face_2d = np.array(face_2d, dtype=np.float64)
        face_3d = np.array(face_3d, dtype=np.float64)
        success, rot_vec, trans_vec = cv.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
        rmat, jac = cv.Rodrigues(rot_vec)
        angles, mtxR, mtxQ, Qx, Qy, Qz = cv.RQDecomp3x3(rmat)
        x = angles[0] * 360
        y = angles[1] * 360
        p1 = (int(nose_2d[0]), int(nose_2d[1]))
        #p2 = (int(nose_3d_projection[0][0][0]), int(nose_3d_projection[0][0][1]))
        p2 = (int(nose_2d[0] + y * 5), int(nose_2d[1] - x * 5))
        x1 = p2[0]
        y1 = p2[1]
        
        ratio, reRatio, leRatio = blinkRatio(mesh_coords, RIGHT_EYE, LEFT_EYE)
        #########################################
        if caliTimer < 120 :
            cv.putText(frame, 'OPEN YOUR EYES', (50, 160), FONTS, 1, RED, 2)
            if caliTimer > 60:
                cv.putText(frame, 'CALIBRATING', (50, 200), FONTS, 1, RED, 2)
                eyeRatioValues = np.append(eyeRatioValues, (reRatio + leRatio/2))
                config["EYERATIO"] = np.nanmax(eyeRatioValues) + 1
                ################################################################ TODO: implement saving to the config file
            caliTimer += 1
        #########################################
        if (clickingMode == 1):
            #cv.putText(frame, 'Right ratio %.2f' % leRatio, (10, 320), FONTS, 0.5, GREEN, 1)
            #cv.putText(frame, 'Left ratio %.2f' % reRatio, (10, 340), FONTS, 0.5, GREEN, 1)
            #cv.putText(frame, '%d' % CEF_COUNTER, (250, 50), FONTS, 1, GREEN, 2)
            if reRatio > config["EYERATIO"] and leRatio > config["EYERATIO"]:
                if (eye_side != 'both'):
                    if (eye_side == 'left' and CEF_COUNTER > CLOSED_EYES_FRAME and (not SCROLL_MODE)):
                        Mouse.release(Button.left)
                    CEF_COUNTER = 0
                    eye_side = 'both'
                    #clicked()
                    blinkTimer.blinking(timeUse)
                    blinkTimer.checkEnd()
                CEF_COUNTER += 1
                if (CEF_COUNTER/fps > 3):
                    SCROLL_MODE = not SCROLL_MODE
                    CEF_COUNTER = 0
                cv.putText(frame, 'both eye close', (375, 50), FONTS, 1, RED, 2)
            elif reRatio > config["EYERATIO"]:
                if (eye_side != 'left'):
                    CEF_COUNTER = 0
                    eye_side = 'left'
                if (CEF_COUNTER == CLOSED_EYES_FRAME and (not SCROLL_MODE)):
                    Mouse.press(Button.left)
                CEF_COUNTER += 1
                cv.putText(frame, 'left eye close', (375, 50), FONTS, 1, RED, 2)
            elif leRatio > config["EYERATIO"]:
                if (eye_side != 'right'):
                    if (eye_side == 'left' and CEF_COUNTER > CLOSED_EYES_FRAME and (not SCROLL_MODE)):
                        Mouse.release(Button.left)
                    CEF_COUNTER = 0
                    eye_side = 'right'
                CEF_COUNTER += 1
                cv.putText(frame, 'right eye close', (375, 50), FONTS, 1, RED, 2)
            else:
                if CEF_COUNTER > CLOSED_EYES_FRAME:
                    if (eye_side != "both" and not SCROLL_MODE):
                        if (eye_side == 'left'):
                            Mouse.release(Button.left)
                        else:
                            Mouse.click(Button.right, 1)
                CEF_COUNTER = 0
                eye_side = ''
        elif (clickingMode == 2):
            rollRatio = RollAngle(mesh_coords[10], mesh_coords[152])
            if rollRatio > ROLLRATIO:
                if (eye_side != 'left'):
                    CEF_COUNTER = 0
                    eye_side = 'left'
                if (CEF_COUNTER == CLOSED_EYES_FRAME and (not SCROLL_MODE)):
                    Mouse.press(Button.left)
                CEF_COUNTER += 1
                cv.putText(frame, 'left turn', (375, 50), FONTS, 1, RED, 2)
            elif rollRatio < -ROLLRATIO:
                if (eye_side != 'right'):
                    if (eye_side == 'left' and CEF_COUNTER > CLOSED_EYES_FRAME and (not SCROLL_MODE)):
                        Mouse.release(Button.left)
                    CEF_COUNTER = 0
                    eye_side = 'right'
                CEF_COUNTER += 1
                cv.putText(frame, 'right turn', (375, 50), FONTS, 1, RED, 2)
            else:
                if CEF_COUNTER > CLOSED_EYES_FRAME:
                    if (not SCROLL_MODE):
                        if (eye_side == 'left'):
                            Mouse.release(Button.left)
                        else:
                            Mouse.click(Button.right, 1)
                CEF_COUNTER = 0
                eye_side = ''
            if reRatio > config["EYERATIO"] and leRatio > config["EYERATIO"]:
                if (not closeEyes):
                    eye_counter = 0
                    closeEyes = True
                    #clicked()
                    blinkTimer.blinking(timeUse)
                    blinkTimer.checkEnd()
                eye_counter += 1
                if (eye_counter/fps > 3):
                    SCROLL_MODE = not SCROLL_MODE
                    eye_counter = 0
                cv.putText(frame, 'both eye close', (375, 50), FONTS, 1, RED, 2)
            else:
                eye_counter = 0
                closeEyes = False

        cv.polylines(frame, [np.array([mesh_coords[p][:2] for p in LEFT_EYE], dtype=np.int32)], True, GREEN, 1,
                     cv.LINE_AA)
        cv.polylines(frame, [np.array([mesh_coords[p][:2] for p in RIGHT_EYE], dtype=np.int32)], True, GREEN, 1,
                     cv.LINE_AA)
        cv.polylines(frame, [np.array([mesh_coords[p][:2] for p in FACE_OVAL], dtype=np.int32)], True, faceColor, 1,
                         cv.LINE_AA)
        #cv.circle(frame, (mesh_coords[10][:2]), 2, (0, 255, 0), -1)
        #cv.circle(frame, (mesh_coords[152][:2]), 2, (255, 0, 0), -1)
        #cv.putText(frame, '%.2f' % RollAngle(mesh_coords[10], mesh_coords[152]), (250, 50), FONTS, 1, GREEN, 2)
        clocx = 0
        clocy = 0

        # 7. Move Mouse
        if (not SCROLL_MODE):
            cv.putText(frame, 'SCROLLING: OFF', (10, 25), FONTS, 0.8, RED, 2)
            #changeText(L4, "SCROLLING: OFF")
            if (MODEL_SELECTION == 1 or clickingMode == 2):
                cv.rectangle(frame, config["start1"], config["end1"], PURPLE, 2)
                if (x1 > config["end1"][0]):
                    clocx = x1 - config["end1"][0]
                elif (x1 < config["start1"][0]):
                    clocx = x1 - config["start1"][0]
                if (y1 > config["end1"][1]):
                    clocy = y1 - config["end1"][1]
                elif (y1 < config["start1"][1]):
                    clocy = y1 - config["start1"][1]
                clocx *= moveSoftening
                clocy *= moveSoftening
                Mouse.move(round(clocx), round(clocy))
            else:
                cv.rectangle(frame, config["start2"], config["end2"], BLUE, 2)
                clocx = np.interp(x1, (config["start2"][0], config["end2"][0]), (-length, wScr + length))
                clocy = np.interp(y1, (config["start2"][1], config["end2"][1]), (-length, hScr + length))
                clocx = EMA(clocx, smoothMouse[0], 5)
                clocy = EMA(clocy, smoothMouse[1], 5)
                smoothMouse[0] = clocx
                smoothMouse[1] = clocy
                update_mouse_freestyle(round(clocx), round(clocy))
                Mouse.position = (mousex, mousey)
        else:
            cv.putText(frame, 'SCROLLING: ON', (10, 25), FONTS, 0.8, GREEN, 2)
            #changeText(L4, "SCROLLING: ON")
            cv.rectangle(frame, config["start1"], config["end1"], PURPLE, 2)
            if (x1 > config["end1"][0]):
                clocx = x1 - config["end1"][0]#plocx + ((x1 - end1[0]) - plocx) // 10
            elif (x1 < config["start1"][0]):
                clocx = x1 - config["start1"][0]#plocx + ((x1 - start1[0]) - plocx) // 10
            if (y1 > config["end1"][1]):
                clocy = y1 - config["end1"][1]#plocy + ((y1 - end1[1]) - plocy) // 10
            elif (y1 < config["start1"][1]):
                clocy = y1 - config["start1"][1]#plocy + ((y1 - start1[1]) - plocy) // 10
            if (clocx != 0):
                if (scrollFrameX > abs(scrollSoftening / clocx)):
                    Mouse.scroll(numSign(clocx), 0)
                    scrollFrameX = 0
                scrollFrameX += 1
            if (clocy != 0):
                if (scrollFrameY > abs(scrollSoftening / clocy)):
                    Mouse.scroll(0, numSign(-clocy))
                    scrollFrameY = 0
                scrollFrameY += 1

        #line
        plocx = clocx
        plocy = clocy
        cv.line(frame, p1, p2, PURPLE, 2)
    elif (not first_out):
        first_out = 1

    # calculating  frame per seconds FPS
    cTime = time.time()
    fps = EMA(1 / (cTime - pTime), fps, 15)
    pTime = cTime
    #cv.putText(frame, '%.2f fps'%fps, (50, 50), FONTS, 0.8, GREEN, 2)    
    
    # update opencv
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    #resized
    frame = cv.resize(frame , (app_width,int(hCam/wCam*app_width)))
    img_update = ImageTk.PhotoImage(Image.fromarray(frame))
    button_image.configure(image=img_update)
    button_image.image = img_update
    
    if PROG_MODE == False:
        on_closing()
    else:
        main_win.after(1, frameUpdate)
    return (0)

main_win.protocol("WM_DELETE_WINDOW", on_closing)

frameUpdate()
main_win.mainloop()

close
